# 🚀 Cafe24 CRM Fine-tuning & A/B/C 실험

**발표용 데이터 생성 노트북**

1. Llama 3 8B QLoRA Fine-tuning (무료 T4 GPU)
2. A/B/C 비교 실험 실행
3. 결과 JSON 다운로드

⏱️ **예상 시간: 30-40분**

## 0️⃣ GPU 확인 & 라이브러리 설치

In [ ]:
# GPU 확인
!nvidia-smi

In [ ]:
# Unsloth 설치 (빠른 fine-tuning)
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes triton
!pip install datasets huggingface_hub

## 1️⃣ 학습 데이터 업로드

**👆 왼쪽 파일 아이콘 클릭 → `cafe24_finetuning_dataset.jsonl` 업로드**

In [ ]:
# 파일 업로드 (수동)
from google.colab import files
import os

# 이미 업로드했으면 스킵
if not os.path.exists('cafe24_finetuning_dataset.jsonl'):
    print("📁 cafe24_finetuning_dataset.jsonl 파일을 업로드하세요!")
    uploaded = files.upload()
else:
    print("✅ 데이터 파일 확인됨!")

# 데이터 확인
!wc -l cafe24_finetuning_dataset.jsonl
!head -1 cafe24_finetuning_dataset.jsonl | python -m json.tool | head -20

## 2️⃣ 모델 로드 (Llama 3 8B + QLoRA)

In [ ]:
from unsloth import FastLanguageModel
import torch

# 모델 설정
max_seq_length = 2048
dtype = None  # auto
load_in_4bit = True  # QLoRA (메모리 절약)

# Llama 3 8B 로드
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

print("✅ Llama 3 8B 로드 완료!")

In [ ]:
# LoRA 설정
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

print("✅ LoRA 적용 완료!")

## 3️⃣ 데이터셋 준비

In [ ]:
from datasets import load_dataset

# JSONL 로드
dataset = load_dataset('json', data_files='cafe24_finetuning_dataset.jsonl', split='train')
print(f"📊 학습 데이터: {len(dataset)}개")

# ChatML 형식으로 변환
def format_prompts(examples):
    texts = []
    for messages in examples['messages']:
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(text)
    return {"text": texts}

dataset = dataset.map(format_prompts, batched=True)
print("✅ 데이터셋 준비 완료!")

## 4️⃣ Fine-tuning 실행 (약 15-20분)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,  # 빠른 학습 (데모용)
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        output_dir="outputs",
    ),
)

print("🚀 Fine-tuning 시작...")
trainer_stats = trainer.train()
print("✅ Fine-tuning 완료!")

## 5️⃣ A/B/C 비교 실험

In [ ]:
import json
import time
from datetime import datetime

# 테스트 프롬프트
TEST_PROMPTS = [
    {
        "id": "customer_lookup",
        "prompt": "Cafe24 고객 조회 API를 사용해서 특정 고객의 정보를 가져오는 방법을 알려줘.",
        "expected": ["customers", "GET", "member_id", "API"]
    },
    {
        "id": "order_status",
        "prompt": "주문 상태를 변경하는 API 호출 코드를 Python으로 작성해줘.",
        "expected": ["orders", "PUT", "order_id", "status"]
    },
    {
        "id": "product_inventory",
        "prompt": "상품 재고를 업데이트하는 API 사용법을 설명해줘.",
        "expected": ["products", "variants", "stock", "quantity"]
    },
    {
        "id": "coupon_create",
        "prompt": "VIP 고객에게 쿠폰을 발급하는 코드를 작성해줘.",
        "expected": ["coupons", "member", "POST", "discount"]
    },
    {
        "id": "mileage_add",
        "prompt": "고객에게 적립금을 지급하는 API 호출 방법을 알려줘.",
        "expected": ["mileage", "point", "member", "POST"]
    }
]

# 시스템 프롬프트
SYSTEM_PROMPTS = {
    "A": """당신은 Cafe24 CRM API 전문가입니다.
다음 온톨로지 컨텍스트를 참고하세요:
- Customer: 고객 정보 (PLACES → Order, HAS_ADDRESS → Address)
- Order: 주문 정보 (CONTAINS → OrderItem, PAID_BY → Payment)
- Product: 상품 정보 (BELONGS_TO → Category)
- Campaign: 마케팅 캠페인 (TARGETS → CustomerSegment)
API 베이스: https://{mall_id}.cafe24api.com/api/v2/admin""",
    
    "B": "당신은 Cafe24 CRM API 전문가입니다.",  # Fine-tuned (학습된 지식 사용)
    
    "C": "당신은 도움이 되는 AI 어시스턴트입니다."  # Vanilla
}

print("✅ 테스트 설정 완료!")

In [ ]:
# 추론 모드로 전환
FastLanguageModel.for_inference(model)

def generate_response(system_prompt, user_prompt):
    """모델 응답 생성"""
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")
    
    start_time = time.time()
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    latency = (time.time() - start_time) * 1000
    
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    return response, latency

def score_response(response, expected):
    """응답 점수 계산"""
    response_lower = response.lower()
    found = sum(1 for e in expected if e.lower() in response_lower)
    relevance = found / len(expected) if expected else 0
    
    accuracy_signals = [
        "http" in response_lower or "api" in response_lower,
        "```" in response,
        "def " in response or "function" in response_lower,
        "cafe24" in response_lower
    ]
    accuracy = sum(accuracy_signals) / len(accuracy_signals)
    
    return relevance, accuracy

print("✅ 추론 함수 준비 완료!")

In [ ]:
# A/B/C 실험 실행
results = []

print("🧪 A/B/C 실험 시작...")
print("="*60)

for test in TEST_PROMPTS:
    print(f"\n📋 테스트: {test['id']}")
    
    for approach in ["A", "B", "C"]:
        print(f"  → 접근법 {approach} 실행 중...", end=" ")
        
        try:
            response, latency = generate_response(
                SYSTEM_PROMPTS[approach],
                test["prompt"]
            )
            relevance, accuracy = score_response(response, test["expected"])
            
            result = {
                "approach": approach,
                "approach_name": {"A": "Ontology+Context", "B": "Fine-tuned", "C": "Vanilla"}[approach],
                "prompt_id": test["id"],
                "prompt": test["prompt"],
                "response": response[:1000],  # 저장용 truncate
                "latency_ms": round(latency, 2),
                "relevance_score": round(relevance, 3),
                "accuracy_score": round(accuracy, 3),
                "combined_score": round(relevance * 0.6 + accuracy * 0.4, 3),
                "timestamp": datetime.now().isoformat()
            }
            results.append(result)
            
            print(f"✅ (관련성: {relevance:.0%}, 정확도: {accuracy:.0%})")
            
        except Exception as e:
            print(f"❌ 오류: {e}")
            results.append({
                "approach": approach,
                "prompt_id": test["id"],
                "error": str(e)
            })

print("\n" + "="*60)
print("✅ 실험 완료!")

## 6️⃣ 결과 분석 & 저장

In [ ]:
# 접근법별 집계
print("\n📊 실험 결과 요약")
print("="*60)

report = {
    "timestamp": datetime.now().isoformat(),
    "total_tests": len(results),
    "approaches": {}
}

for approach in ["A", "B", "C"]:
    approach_results = [r for r in results if r.get("approach") == approach and "error" not in r]
    
    if approach_results:
        avg_latency = sum(r["latency_ms"] for r in approach_results) / len(approach_results)
        avg_relevance = sum(r["relevance_score"] for r in approach_results) / len(approach_results)
        avg_accuracy = sum(r["accuracy_score"] for r in approach_results) / len(approach_results)
        avg_combined = sum(r["combined_score"] for r in approach_results) / len(approach_results)
        
        name = {"A": "Ontology+Context", "B": "Fine-tuned", "C": "Vanilla"}[approach]
        
        report["approaches"][approach] = {
            "name": name,
            "avg_latency_ms": round(avg_latency, 2),
            "avg_relevance": round(avg_relevance, 3),
            "avg_accuracy": round(avg_accuracy, 3),
            "avg_combined": round(avg_combined, 3),
            "test_count": len(approach_results)
        }
        
        print(f"\n{approach}: {name}")
        print(f"  평균 응답시간: {avg_latency:.0f}ms")
        print(f"  관련성 점수: {avg_relevance:.1%}")
        print(f"  정확도 점수: {avg_accuracy:.1%}")
        print(f"  종합 점수: {avg_combined:.1%}")

# 우승자 결정
scores = {k: v["avg_combined"] for k, v in report["approaches"].items()}
winner = max(scores, key=scores.get)
report["winner"] = winner
report["winner_name"] = report["approaches"][winner]["name"]

print("\n" + "="*60)
print(f"🏆 우승: {winner} ({report['winner_name']}) - 종합 {scores[winner]:.1%}")

In [ ]:
# 결과 저장
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# 상세 결과
results_file = f"abc_results_{timestamp}.json"
with open(results_file, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

# 요약 리포트
report_file = f"abc_report_{timestamp}.json"
with open(report_file, "w", encoding="utf-8") as f:
    json.dump(report, f, ensure_ascii=False, indent=2)

print(f"\n📁 저장 완료:")
print(f"  - {results_file}")
print(f"  - {report_file}")

In [ ]:
# 파일 다운로드
from google.colab import files

print("📥 결과 파일 다운로드:")
files.download(results_file)
files.download(report_file)

## ✅ 완료!

다운로드된 파일:
- `abc_results_*.json` - 상세 실험 결과
- `abc_report_*.json` - 요약 리포트

이 파일들을 `/Users/admin/cafe24-crm-prototype/experiments/` 에 저장하세요.